In [11]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [12]:
import os
from utils.preprocessing import load_tif_images, flatten_bands, scale_features
import matplotlib.pyplot as plt
import numpy as np

# Load raw images
images, paths = load_tif_images("C:/Users/dell/Documents/GitHub/satellite-image-analysis/data/sample")
print(f"Loaded {len(images)} images.")

ModuleNotFoundError: No module named 'utils.preprocessing'

## K-means Clustering

In [ ]:
from sklearn.cluster import KMeans

# Example: cluster the first image
sample_image = images[0]
features = flatten_bands(sample_image)
features_scaled = scale_features(features)

# Apply KMeans
kmeans = KMeans(n_clusters=5, random_state=42)
labels = kmeans.fit_predict(features_scaled)

# Reshape labels back to image shape
height, width = sample_image.shape[1], sample_image.shape[2]
clustered_image = labels.reshape((height, width))

# Visualize
plt.imshow(clustered_image, cmap='tab20')
plt.title("K-Means Clustering")
plt.axis('off')
plt.show()
